In [187]:
# anchors = [[5,5,5],[10,10,10],[15,15,15],[20,20,20],[30,30,30],[5,5,2.5],[10,10,5],[15,15,7.5],[20,20,10],[30,30,15]]
# anchors = [[5,5,5],[10,10,10],[15,15,15],[20,20,20],[30,30,30]]
# anchors = [[6,6,6],[12,12,12],[24,24,24],[9,9,6],[18,18,12],[36,36,24]]
anchors = [[5,5,3],[8,8,4.8],[14,14,8.4],[28,28,16.8]]

In [227]:
import itertools
import numpy as np


def make_rpn_windows(f_shape):
    """
    Generating anchor boxes at each voxel on the feature map,
    the center of the anchor box on each voxel corresponds to center
    on the original input image.

    return
    windows: list of anchor boxes, [z, y, x, d, h, w]
    """
    stride = 4
    anchors = np.asarray(anchors)
    offset = (float(stride) - 1) / 2
    _, _, D, H, W = f_shape
    oz = np.arange(offset, offset + stride * (D - 1) + 1, stride)
    oh = np.arange(offset, offset + stride * (H - 1) + 1, stride)
    ow = np.arange(offset, offset + stride * (W - 1) + 1, stride)

    windows = []
    for z, y, x, a in itertools.product(oz, oh , ow , anchors):
        windows.append([z, y, x, a[0], a[1], a[2]])
    windows = np.array(windows)

    return windows

def find_nearest(x,y,z):
    stride = 4
    start = (stride-1) / 2
    z_stride = 4
    z_start = (z_stride-1) / 2
    xl = ((x - start) // stride) *stride + start
    xr = xl+stride
    xn = xl if x-xl < xr-x else xr

    yl = ((y - start) // stride) *stride + start
    yr = yl+stride
    yn = yl if y-yl < yr-y else yr

    zl = ((z - z_start) // z_stride) *z_stride + z_start
    zr = zl+z_stride
    zn = zl if z-zl < zr-z else zr

    return xn, yn, zn

def IoU(bbox1, bbox2):
    intersection_min = np.maximum(bbox1[:3], bbox2[:3])
    intersection_max = np.minimum(bbox1[3:], bbox2[3:])
    intersection_dims = np.maximum(0, intersection_max - intersection_min)

    intersection_volume = np.prod(intersection_dims)

    volume1 = np.prod(bbox1[3:] - bbox1[:3])
    volume2 = np.prod(bbox2[3:] - bbox2[:3])
    union_volume = volume1 + volume2 - intersection_volume

    iou = intersection_volume / union_volume if union_volume > 0 else 0
    return iou


def distance(cord1, cord2):
    return ((cord1 - cord2)**2).sum()

def DIoU(cord1, shape1, cord2, shape2):
    box1 = np.array([*cord1-shape1/2, *cord1+shape1/2])
    box2 = np.array([*cord2-shape2/2, *cord2+shape2/2])

    c1 = np.amax([box1,box2],axis=0)[3:]
    c2 = np.amin([box1,box2],axis=0)[:3]

    return IoU(box1, box2) - distance(cord1, cord2)/distance(c1, c2)

In [208]:
cord = np.array([35.7334   , 22.434937 , 51.367676])
shape = np.array([12.902344 ,  9.325195 ,         7.998047 ])
[*cord-shape/2, *cord+shape/2]

[29.282228000000003,
 17.7723395,
 47.3686525,
 42.184572,
 27.097534500000002,
 55.3666995]

In [233]:
target=[[ 0      ,  0.       ,  0.       ,   4,  4.       ,         4 ],
        [ 88.5      ,  66.       ,  96.       ,  10.       ,  13.       ,         12.999999 ],
        [ 88.5      ,  66.       ,  96.       ,  10.       ,  13.       ,         12.999999 ],
        [ 88.5      ,  66.       ,  96.       ,  10.       ,  12.999999 ,         12.999999 ]]
pred = [[ 2      ,  2.       ,  2.       ,   4,  4.       ,         4 ],
        [ 88.5      ,  66.       ,  96.       ,  10.       ,  13.       ,         12.999999 ],
        [ 88.5      ,  66.       ,  96.       ,  10.       ,  13.       ,         12.999999 ],
        [ 88.5      ,  66.       ,  96.       ,  10.       ,  12.999999 ,         12.999999 ]]
target = np.array(target)
pred = np.array(pred)

# DIoU(p[:,:3], p[:,3:], t[:,:3], t[:,3:])
for p ,t in zip(pred, target):
    print(DIoU(p[:3], p[3:],t[:3], t[3:]))

0.06666666666666667
-0.04444444444444444
1.0
1.0
1.0
1.0
1.0
1.0


In [190]:
import json
import os
nodule_distribution = [0,0,0,0]
anchor_distribution = [0 for i in range(len(anchors))]

train_txt = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_train.txt'
with open(train_txt, 'r') as f:
    train_files = f.readlines()
anchor_count = [0,0,0]
bad_nodule = []
for train_file in train_files[1:]:
    p1, p2 = train_file.replace('\n','').split(',')

    dicom_nodule_path = os.path.join(p1, 'mask', f'{p2}_nodule_count.json') # CHEST1001_nodule_count
    with open(dicom_nodule_path, 'r') as f:
        dicom_nodule = json.load(f)
    nodules = dicom_nodule['bboxes']
    for nodule in nodules:
        good_nodule = 0
        cx = (nodule[0][0]+nodule[1][0])/2
        cy = (nodule[0][1]+nodule[1][1])/2
        cz = (nodule[0][2]+nodule[1][2])/2
        w = (nodule[1][0]-nodule[0][0])+4
        h = (nodule[1][1]-nodule[0][1])+4
        d = (nodule[1][2]-nodule[0][2])+4

        size = (4/3)*3.1415*(h*w*d / 6)
        if size <= 52:
            size_idx = 0
        elif size <= 113:
            size_idx = 1
        elif size <= 268:
            size_idx = 2
        else:
            size_idx = 3

        nodule_distribution[size_idx] += 1

        nearest_point = find_nearest(cx, cy, cz)

        for i, anchor in enumerate(anchors):
            iou = IoU([cx,cy,cz], [w,h,d], nearest_point, anchor)
            if iou >= 0.3:
                anchor_distribution[i] += 1
                good_nodule = 1
                continue
            elif iou >= 0.1 and good_nodule < 1:
                good_nodule = 2

        anchor_count[good_nodule] += 1
        if not good_nodule:
            bad_nodule.append([size_idx, w,h,d])
print(anchor_distribution, '\n', nodule_distribution, '\n', anchor_count)

[3, 845, 1475, 84] 
 [0, 3, 46, 1761] 
 [1, 1782, 27]


In [166]:
bad_nodule

[[3, 7, 8, 7]]

In [136]:
a = '318.56763	334.10632	108.51063	8.093452	8.115494	5.353752'
b = '318.83087	334.1272	102.111305	8.714816	7.911281	6.0994024'
a = a.split('\t')
a_cord = [float(a_) for a_ in a[:3]]
a_shape = [float(a_)+2. for a_ in a[3:]]
b = b.split('\t')
b_cord = [float(b_) for b_ in b[:3]]
b_shape = [float(b_)+2. for b_ in b[3:]]
IoU(a_cord, a_shape, b_cord, b_shape)

0.08982469602420033

In [96]:
a_shape, b_shape

([9.231955, 9.961705, 7.010294], [10.86131, 10.294744, 7.4044294])

In [41]:
sub_path = r'F:\master\code\LSSANet-main\MsaNet_R_results\ME_LDCT\AdamW0.0003_Bs6x4_OHEM10_bb0\res\72\FROC\submission_rpn.csv'
gt_path  = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_test_annotation.csv'

import csv
sub_by_file = {}
with open(sub_path, newline='') as fs:
    subs = csv.reader(fs,)
    for sub in subs:
        if sub[0] in sub_by_file.keys():
            sub_by_file[sub[0]].append([float(xyzhwd) for xyzhwd in sub[1:8]])
        elif sub[0] != 'series_id':
            sub_by_file[sub[0]] = [[float(xyzhwd) for xyzhwd in sub[1:8]]]

gt_by_file  = {}
with open(gt_path, newline='') as fg:
    gts = csv.reader(fg)
    for gt in gts:
        if gt[0] in gt_by_file.keys():
            gt_by_file[gt[0]].append([float(xyzhwd) for xyzhwd in gt[1:8]])
        elif gt[0] != 'series_id':
            gt_by_file[gt[0]] = [[float(xyzhwd) for xyzhwd in gt[1:8]]]

files = list(gt_by_file.keys())


In [26]:
TP = []
FN = []
gt_IoU = []
gt_DIoU = []
for file in files:
    gts = gt_by_file[file]
    subs = sub_by_file[file]
    for gt in gts:
        is_cand = 0
        re_detected = 0
        max_iou = 0.0
        max_diou = 0.0
        gt_cord, gt_shape = gt[:3], gt[3:6]
        for sub in subs:
            iou = IoU(gt_cord, gt_shape, sub[:3], sub[3:6])
            if iou >= 0.1:
                if is_cand:
                    re_detected += 1
                is_cand = 1
            max_iou = max(max_iou, iou)
            max_diou = max(max_diou, DIoU(gt_cord, gt_shape, sub[:3], sub[3:6]))
        gt_IoU.append([max_iou, max_diou])
        gt_DIoU.append(max_diou)
        if is_cand:
            TP.append(max_iou)
        else:
            FN.append(max_iou)
                

In [45]:
import os
import numpy as np
import warnings
import json
from single_config import config
from scipy.ndimage import zoom

def crop_with_lobe(z:str, yx:str):
    zlobe = z.replace('\n','').split(',')
    Ds, De = [int(z_) for z_ in zlobe]
    yxlobe = yx.replace('\n','').split(',')
    Hs, He, Ws, We = [int(yx_) for yx_ in yxlobe]
    
    # align to 16
    align = 16
    Ds = (Ds//align)*align
    De = ((De+align-1)//align)*align
    Hs = (Hs//align)*align
    He = ((He+align-1)//align)*align
    Ws = (Ws//align)*align
    We = ((We+align-1)//align)*align

    return (Ds, De, Hs, He, Ws, We)

def load_img(filename):
    path, dir = filename.split(',')
    img = np.load('%s\\npy\\%s.npy' % (path, dir))
    # img = img[np.newaxis,...] # (y, x, z) -> (1, y, x, z)
    ## load lobe info
    with open('%s\\npy\\lobe_info.txt' %(path)) as f:
        lobe_info = f.readlines()[-2:]
    with open('%s\\mask\\%s_nodule_count.json' % (path, dir)) as f:
        nodule_count = json.load(f)
    bboxes = nodule_count['bboxes']
    Ds, De, Hs, He, Ws, We = crop_with_lobe(*lobe_info)
    # crop the lobe
    img = img[np.newaxis, Hs:He, Ws:We, Ds:De]

    img = np.clip(img, -1000, 400)
    img = img.astype(np.float32)
    img = img.transpose(0, 3, 1, 2) # (1, y, x, z) -> (1, z, y, x)
    images = img + 1000    # 0 ~ max
    images = (images - 700) / 700 # -1 ~ 1
    return images, (Ds, De, Hs, He, Ws, We), bboxes

class Crop(object):
    def __init__(self, config):
        self.crop_size = config['crop_size']
        self.bound_size = config['bound_size']
        self.stride = config['stride']
        self.pad_value = config['pad_value']

    def __call__(self, imgs, target, bboxes, lobe=None, isScale=False, isRand=False):
        '''
        img: 3D image loading from npy, (1, d, h, w)
        target: one nodule
        bboxes: all nodules in series
        '''
        if isScale:
            radiusLim = [8.,120.]
            scaleLim = [0.75,1.25]
            scaleRange = [np.min([np.max([(radiusLim[0]/target[3]),scaleLim[0]]),1])
                         ,np.max([np.min([(radiusLim[1]/target[3]),scaleLim[1]]),1])]
            scale = np.random.rand()*(scaleRange[1]-scaleRange[0])+scaleRange[0]
            crop_size = (np.array(self.crop_size).astype('float')/scale).astype('int')
        else:
            crop_size = self.crop_size
        bound_size = self.bound_size
        target = np.copy(target)
        bboxes = np.copy(bboxes)
        start = []
        for i in range(3):
            # start.append(int(target[i] - crop_size[i] / 2))
            if not isRand:
                # crop the sample base on target
                r = target[i+3]/2
                s = np.floor(target[i] - r) + 1 - bound_size
                e = np.ceil(target[i] + r) + 1 + bound_size - crop_size[i]
            else:
                # crop the sample randomly
                s = np.max([imgs.shape[i+1]-crop_size[i]/2, imgs.shape[i+1]/2+bound_size])
                e = np.min([crop_size[i]/2, imgs.shape[i+1]/2-bound_size])
                target = np.array([np.nan, np.nan, np.nan, np.nan])
            if s > e:
                i_start = np.random.randint(e, s)
                i_start = max(min(i_start, imgs.shape[i+1]-crop_size[i]),0)
                start.append(i_start)#!
            else:
                start.append(int(target[i])-crop_size[i]/2+np.random.randint(-bound_size/2,bound_size/2))

        coord=[]
        pad = []
        pad.append([0,0])

        for i in range(3):
            leftpad = max(0,-start[i]) # how many pixel need to pad on the left side
            rightpad = max(0,start[i]+crop_size[i]-imgs.shape[i+1]) # how many pixel need to pad on the right side
            pad.append([leftpad,rightpad])
            
        crop = imgs[:,
            int(max(start[0],0)):int(min(start[0] + int(crop_size[0]),imgs.shape[1])),
            int(max(start[1],0)):int(min(start[1] + int(crop_size[1]),imgs.shape[2])),
            int(max(start[2],0)):int(min(start[2] + int(crop_size[2]),imgs.shape[3]))]

        crop = np.pad(crop, pad, 'constant', constant_values=0.67142)
        for i in range(3):
            target[i] = target[i] - start[i]
        for i in range(len(bboxes)):
            for j in range(3):
                bboxes[i][j] = bboxes[i][j] - start[j]

        if isScale:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                crop = zoom(crop, [1, scale, scale, scale], order=1)
            newpad = self.crop_size[0] - crop.shape[1:][0]
            if newpad < 0:
                crop = crop[:, :-newpad, :-newpad, :-newpad]
            elif newpad > 0:
                pad2 = [[0, 0], [0, newpad], [0, newpad], [0, newpad]]
                crop = np.pad(crop, pad2, 'constant', constant_values=0.67142)
            for i in range(6):
                target[i] = target[i] * scale
            for i in range(len(bboxes)):
                for j in range(6):
                    bboxes[i][j] = bboxes[i][j] * scale

        return crop, target, bboxes, coord

crop = Crop(config)

In [ ]:
test_txt = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_test.txt'
with open(test_txt, 'r') as f:
    files = f.readlines()
imgs, lobe_info, bboxes = load_img(files[1])
bboxes = bboxes - [lobe_info[0], lobe_info[2], lobe_info[4], 0, 0, 0]
for bbox in bboxes:
    sample, target, bboxes, coord = crop(imgs, bbox, bboxes, isScale=False, isRand=False)
    

In [123]:
import os
import json
import numpy as np
from utils.util import crop_with_lobe
test_dicom_npy = r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014\npy\CHESTCT_Test0014.npy'
test_dicom_info = r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014\mask\CHESTCT_Test0014_nodule_count.json'
test_dicom_lobe = r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014\npy\lobe_info.txt'

filename = [r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014', 'CHESTCT_Test0014']
path, dir = filename
img = np.load(os.path.join('%s\\npy\\%s.npy' % (path, dir)))
img = img[np.newaxis,...] # (y, x, z) -> (1, y, x, z)
## load lobe info
with open(os.path.join('%s\\npy\\lobe_info.txt' %(path))) as f:
    # ['z_start,z_end\n',
    #  'y_start,y_end,x_start,x_end']
    lobe_info = f.readlines()[-2:]
    lobes = crop_with_lobe(*lobe_info)

with open(test_dicom_info, 'r') as f:
    dicom_info = json.load(f)
    nodules = dicom_info["bboxes"]
bboxes = []
for nodule in nodules:
    nodule = np.array(nodule)
    cx, cy, cz = (nodule[1,:] + nodule[0,:]+1) /2
    w,  h,  d  = (nodule[1,:] - nodule[0,:]+1)
    bboxes.append(np.array([0.,cz, cy, cx, d, h, w]))
bboxes = np.array(bboxes)

In [121]:
lobes

(16, 288, 128, 400, 64, 448)

In [124]:
bboxes

array([[  0. ,  71.5, 171. , 332.5,   5. ,   6. ,   7. ],
       [  0. , 251.5, 407. , 335. ,   9. ,  10. ,  12. ],
       [  0. , 141. , 185.5, 368.5,   6. ,   7. ,   5. ]])

In [138]:
from utils.pybox import *
import torch
while True:
    overlap_nodule = False
    rz = np.random.randint(lobes[0]+64, lobes[1]-64)
    ry = np.random.randint(lobes[2]+64, lobes[3]-64)
    rx = np.random.randint(lobes[4]+64, lobes[5]-64)
    overlaps = torch_overlap(bboxes[:,-6:], np.array([rz, ry, rx, 128, 128, 128]))
    for overlap in overlaps:
        if overlap[0] > 0.00:
            overlap_nodule = True
            break
    if not overlap_nodule:
        break
    else:
        print(f'{rz} {ry} {rx} overlapped with nodule')
print(rz, ry, rx)



87 214 364 overlapped with nodule
81 248 223


In [73]:
Ds, De, Hs, He, Ws, We = crop_with_lobe(*lobe_info)
# crop the lobe
img = img[np.newaxis, Hs:He, Ws:We, Ds:De]

img = np.clip(img, -1000, 400)
img = img.astype(np.float32)
img = img.transpose(0, 3, 1, 2) # (1, y, x, z) -> (1, z, y, x)
images = img + 1000    # 0 ~ max
images = (images - 700) / 700 # -1 ~ 1

ValueError: axes don't match array

In [126]:
rz = np.random.randint(lobes[0]+64, lobes[1]-64)
ry = np.random.randint(lobes[2]+64, lobes[3]-64)
rx = np.random.randint(lobes[4]+64, lobes[5]-64)
print(rz, ry, rx)
torch_overlap(bboxes[:,-6:], np.array([rx, ry, rz, 128, 128,128]))

146 308 317


tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [183]:
a = np.array([i for i in range(10)])
p = np.array([0.1 for i in range(10)])

In [184]:
count = np.array([0 for i in range(10)])
for i in range(1000):
    idcs = np.random.choice(len(a), 2, replace=False)
    count[idcs] += 1

random choice

In [185]:
count

array([201, 217, 185, 203, 194, 188, 193, 223, 218, 178])

In [186]:
count2 = np.array([0 for i in range(10)])
for i in range(1000):
    idcs = np.random.choice(len(a), 2, replace=False, p=p)
    p[idcs] *= 0.5
    p /= p.sum()
    count2[idcs]+=1

choice with weight p

In [187]:
count2

array([198, 201, 200, 199, 200, 200, 200, 201, 200, 201])

In [192]:
count.var()

209.0

In [193]:
count2.var()

0.8

In [81]:
import numpy as np
decay = 0.9
w = np.array([0.25,0.25,0.25,0.25])
p = 0.75
pos = 0
neg = 0
for i in range(1000):
    idx = np.random.choice(4,1,p=w)
    if idx <= 2:
        pos +=1
    else:
        neg +=1
        # w *= decay
    w[idx] *= decay
    w /= w.sum()

print(pos, neg)

749 251


In [51]:
w

1.2345679012345678

In [104]:
np.random.rand()

0.049309694441802776

In [73]:
from dataset.bbox_reader_neg_nodulebased import BboxReader_NegNB
from single_config import config, datasets_info
data_dir = r'F:\master\code\Lung_Nodule\dataset\ME_dataset'
set_name = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_train.txt'
augtype = config['augtype']
dataset = BboxReader_NegNB(data_dir, set_name, augtype, config, mode='train')


In [74]:
from torch.utils.data import DataLoader
from dataset.collate import train_collate
train_loader = DataLoader(dataset, batch_size=6, shuffle=True,
                              num_workers=6, pin_memory=True, collate_fn=train_collate, drop_last=True)

In [75]:
loader = iter(train_loader)
data = next(loader)

In [78]:
inputs, box, label = data
inputs.shape

torch.Size([6, 1, 128, 128, 128])

In [ ]:
for j, (input, truth_box, truth_label) in enumerate(train_loader):
    print(truth_label[:,0])

In [22]:
import numpy as np 

if np.random.choice(4, 1, p=[0.25,0.25,0.25,0.25]) < 2:
    print('True')

In [26]:
int(np.round(439.49))

439

In [38]:
tpr = 61.9647355163728
tnr = 99.99930071969138
tol_pos = 1191
tol_neg = 60633768
recall = tpr/100
precision = (tpr*tol_pos) / (tpr*tol_pos + (100-tnr)*tol_neg)
f1 = 2*recall*precision/(recall+precision)
print(recall,'\n',precision,'\n',f1)

0.6196473551637279 
 0.6351118760755053 
 0.6272843178919424


In [141]:
from evaluationScript.tools import csvTools
anno_path = r'F:\master\code\LSSANet-main\MsaNet_R_results\ME_LDCT\AdamW0.0003_Bs6x4_OHEM10_bb0_trueOHEM\res\81\FROC\submission_rpn.csv'
annotations = csvTools.readCSV(anno_path)

In [142]:
dicom_nodules = {}
for anno in annotations[1:]:
    sid, x, y, z, w, h, d, prob = anno
    if sid in dicom_nodules.keys():
        dicom_nodules[sid] = np.append(dicom_nodules[sid],np.array([[float(prob),float(x),float(y),float(z),float(w),float(h),float(d)]]),axis=0)
    else:
        dicom_nodules[sid] = np.array([[float(prob),float(x),float(y),float(z),float(w),float(h),float(d)]])

In [143]:
from utils.pybox import torch_nms
import torch
import pandas as pd
res_anno = []
col_names = ['series_id', 'x_center', 'y_center', 'z_center', 'w_mm', 'h_mm', 'd_mm', 'probability']
rpn_submission_path = r'F:\master\code\LSSANet-main\MsaNet_R_results\ME_LDCT\AdamW0.0003_Bs6x4_OHEM10_bb0_trueOHEM\res\81\FROC\submission_rpn_nms.csv'
for key in dicom_nodules.keys():
    nodules = dicom_nodules[key] + [0,0,0,0,2,2,2]
    _, keep = torch_nms(torch.from_numpy(nodules.astype(np.float32)), 0.1)
    outs = dicom_nodules[key][keep]
    if len(keep) == 1:
        outs = [outs]
    for out in outs:
        res_anno.append([key, str(out[1]), str(out[2]), str(out[3]), str(out[4]), str(out[5]), str(out[6]), str(out[0])])

df = pd.DataFrame(res_anno, columns=col_names)
df.to_csv(rpn_submission_path, index=False)

In [14]:
import numpy as np
offset_ratio = 0.3
r = np.random.rand(3,6)
r[:,:3] = r[:,:3]* (2*offset_ratio) - offset_ratio
r[:,3:] = r[:,3:]* (4*offset_ratio) + (1-2*offset_ratio) 
r

array([[-0.15525517,  0.19141471, -0.21032706,  0.96144958,  1.59379141,
         1.24662869],
       [ 0.13331018, -0.26620126,  0.16050524,  0.82845624,  0.82099605,
         0.77365738],
       [-0.02921804, -0.0392191 ,  0.10936578,  0.99105797,  0.90804561,
         1.33544733]])

In [7]:
np.random.rand(3,6)

array([[0.02037814, 0.29622117, 0.1026084 , 0.06735228, 0.46886298,
        0.3733121 ],
       [0.69613119, 0.74726365, 0.32587972, 0.17928147, 0.36727735,
        0.74830827],
       [0.41786589, 0.50988561, 0.17664829, 0.43305144, 0.36943331,
        0.01211162]])

In [23]:
def ext2factor(arr, factor = 8):
    arr[:,-3:] = (arr[:,-3:]+(factor-1))//factor *factor
    return arr
def 

In [24]:
a = np.array([[  1.0000,   0.8276,  83.1051, 103.0656,  97.2462,   6.0293,   8.1079,  8.2207],
            [  1.0000,   0.8276,  84.2130, 100.7422,  95.9456,   6.3478,   9.6913,   8.7561],
            [  1.0000,   0.8276,  83.1589, 104.3448,  97.2556,   8.9599,  12.2215,   6.3697],
            [  1.0000,   0.8276,  83.6657, 102.6938,  96.9911,   7.3578,   7.4695,  11.9133],
            [  1.0000,   0.7686,  71.2205,  93.9699,  65.5282,   5.8052,   7.8003,   7.9395],
            [  1.0000,   0.7686,  72.3069,  96.1436,  65.8831,   8.1346,   8.0926,  11.9795],
            [  1.0000,   0.7686,  70.7343,  94.8019,  67.8421,   5.7376,   9.0122,   8.2947],
            [  1.0000,   0.7686,  69.8497,  93.8442,  64.0835,   8.2857,  11.9549,  11.9151]])

a = a[:,[0,2,3,4,5,6,7]].astype(np.int32)
a = ext2factor(a, factor=8)

In [25]:
a

array([[  1,  83, 103,  97,   8,   8,   8],
       [  1,  84, 100,  95,   8,  16,   8],
       [  1,  83, 104,  97,   8,  16,   8],
       [  1,  83, 102,  96,   8,   8,  16],
       [  1,  71,  93,  65,   8,   8,   8],
       [  1,  72,  96,  65,   8,   8,  16],
       [  1,  70,  94,  67,   8,  16,   8],
       [  1,  69,  93,  64,   8,  16,  16]])

In [46]:
import torch
import numpy as np
arr = np.random.randn(3,4,6)
tensor = torch.from_numpy(arr)
print(arr, tensor)


[[[ 0.58652611 -0.08603455  1.50398733 -0.36092358  0.56573136
    0.9860972 ]
  [ 0.85878529  1.92066709 -1.3503354  -0.34804173 -0.52167635
    0.46349816]
  [-0.55351909  0.47625656  1.33297321  0.27401318 -1.5684168
   -1.07712571]
  [-1.91666554  0.54255647 -1.09073866  0.07678744 -0.13973057
    1.10318014]]

 [[-0.81955314  0.0975312  -0.78889382 -0.02940587 -1.22535726
    0.46930628]
  [ 0.34801056 -1.74838815  1.40443309 -0.8201392   0.67418694
    0.44477293]
  [ 0.76509467 -2.90221815  0.39322361  0.33112346 -0.17325017
    0.29825359]
  [-1.63673518  1.33871714  0.2017215   0.38491992 -0.25858403
   -1.45034208]]

 [[-0.25509721 -0.99032769 -0.75205615  1.44849796 -1.05677967
    0.04867981]
  [ 0.68567483 -0.02200537 -1.46960665  0.1385924   0.26559375
    0.94404135]
  [ 1.63269586 -0.23848062 -1.13844966 -0.25708323 -1.21329134
   -0.13339221]
  [-0.06256229  0.374907    1.01982907  0.14659279 -0.78167734
   -0.34733871]]] tensor([[[ 0.5865, -0.0860,  1.5040, -0.3609,  

In [69]:
def clip_delta_variance(deltas, threshold):
    deltas = deltas.view(-1,4,6)
    var = torch.var(deltas, dim=1, unbiased=False)
    mean_var = torch.mean(var, dim=1)
    print(var, var.shape, mean_var)
    keeps = torch.where(mean_var<=threshold)
    return deltas[keeps,0], keeps

def clip_delta_variance_np(deltas, threshold):
    deltas = deltas.reshape(-1,4,6)
    var = np.var(deltas, axis=1)
    mean_var = np.mean(var, axis=1)
    print(var, var.shape, mean_var)
    keeps = np.where(mean_var<=threshold)
    return deltas[keeps,0], keeps

In [70]:
clip_delta_variance(tensor, 0.5)

tensor([[1.1997, 0.5455, 1.7532, 0.0751, 0.5948, 0.7550],
        [0.9015, 2.6746, 0.6059, 0.2317, 0.4528, 0.6491],
        [0.5510, 0.2467, 0.9231, 0.4150, 0.3324, 0.2416]], dtype=torch.float64) torch.Size([3, 6]) tensor([0.8206, 0.9193, 0.4516], dtype=torch.float64)


(tensor([[-0.2551, -0.9903, -0.7521,  1.4485, -1.0568,  0.0487]],
        dtype=torch.float64),
 (tensor([2]),))

In [71]:
clip_delta_variance_np(arr, 0.5)

[[1.19973099 0.54550349 1.75318262 0.07507698 0.5948383  0.75500972]
 [0.90150385 2.67459574 0.605946   0.23171086 0.45283382 0.64908915]
 [0.55103076 0.24671458 0.92305489 0.41496215 0.33243857 0.24162199]] (3, 6) [0.82055702 0.9192799  0.45163716]


(array([[[-0.25509721, -0.99032769, -0.75205615,  1.44849796,
          -1.05677967,  0.04867981]]]),
 (array([2], dtype=int64),))

In [72]:
a =  [ 0.1327,  0.1902, -0.0953,  0.2576,  0.3372, -0.6992,  0.0238,  0.0277,-0.0214, -0.0285,  0.0082,  0.0133]
a[0::6]

[0.1327, 0.0238]

In [ ]:
labeled_train_dataset = BboxReader_NegNB(data_dir, train_set_list, augtype, config, mode='train')
labeled_train_loader = DataLoader(labeled_train_dataset, batch_size=4, shuffle=True,
                            num_workers=4, pin_memory=True, collate_fn=train_collate, drop_last=True)

In [352]:
import numpy as np
import torch
def swap(arr):
    while True:
        axisorder = torch.randperm(3)
        if not torch.all(axisorder == torch.tensor([0, 1, 2])):
            break
    axisorder = torch.cat([torch.tensor([0,1]),axisorder+2], dim=0)
    arr = arr.permute(*axisorder)
    unswap = [0, 0, 0, 0, 0]
    for i in range(5):
        unswap[axisorder[i]] = i
    return arr, torch.tensor(unswap)

def flip(arr):
    flipid = np.where([0,0,np.random.randint(2),np.random.randint(2),np.random.randint(2)])[0]
    if flipid.sum() == 0:
        flipid = [np.random.randint(2,5)]
        print(flipid)
    
    arr = torch.flip(arr, dims=tuple(flipid))

    return arr, flipid


def unswap(arr, unswap):
    return arr.permute(*unswap)

def unflip(arr, unflip):
    return torch.flip(arr, dims=tuple(unflip)).contiguous()


In [357]:
swap_augment = []
flip_augment = []
scale = 2
a = torch.tensor([[[[[i*(scale**2)+j*scale+k for k in range(scale)] for j in range(scale)] for i in range(scale)]]])
print(a)
for i in range(5):
    a, unswapax = swap(a)
    a, unflipid = flip(a)
    # print(a)
    swap_augment.append(unswapax)
    flip_augment.append(unflipid)

tensor([[[[[0, 1],
           [2, 3]],

          [[4, 5],
           [6, 7]]]]])


In [358]:
flip_augment.reverse()
swap_augment.reverse()
for i in range(5):
    a = unflip(a, flip_augment[i])
    a = unswap(a, swap_augment[i])

In [161]:
a

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

In [276]:
np.where([0,0,np.random.randint(2),np.random.randint(2),np.random.randint(2)])[0].sum()

0

In [311]:
tuple(np.where([0,0,np.random.randint(2),np.random.randint(2),np.random.randint(2)])[0])

()

In [312]:
tuple([np.random.randint(2,5)])

(4,)